In [4]:
import json

def get_key():
    with open("secretkey.json", "r") as json_file:
        return json.load(json_file)["api_key"]


In [12]:
def txtReader(path):
    with open(path, "r") as file:
        return file.read()

print(txtReader("./test_files/question.txt"))

In [57]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader

loader = UnstructuredWordDocumentLoader("./test_files/decrypted.docx", mode="elements")

data = loader.load()
data[0]

Document(metadata={'source': './test_files/decrypted.docx', 'file_directory': './test_files', 'filename': 'decrypted.docx', 'last_modified': '2024-07-29T11:05:40', 'text_as_html': '<table>\n<thead>\n<tr><th>Question 1:  </th><th>Which of the following adaptions to cell injury is described as an increased number of cells within an organ?\t    </th><th>Which of the following adaptions to cell injury is described as an increased number of cells within an organ?\t            </th></tr>\n</thead>\n<tbody>\n<tr><td>             </td><td>   </td><td>           </td></tr>\n<tr><td>             </td><td>(A)</td><td>Atrophy    </td></tr>\n<tr><td>             </td><td>(B)</td><td>Dysplasia  </td></tr>\n<tr><td>             </td><td>(C)</td><td>Hyperplasia</td></tr>\n<tr><td>             </td><td>(D)</td><td>Hypertrophy</td></tr>\n<tr><td>             </td><td>(E)</td><td>Metaplasia </td></tr>\n<tr><td>             </td><td>   </td><td>           </td></tr>\n<tr><td>             </td><td>   </td>

In [53]:

def extract_page_content(content):
    extracted_text = ""
    for document in content:
        extracted_text += document.page_content + "\n" * 5
    return extracted_text


print(extract_page_content(data))

Question 1: Which of the following adaptions to cell injury is described as an increased number of cells within an organ? (A) Atrophy (B) Dysplasia (C) Hyperplasia (D) Hypertrophy (E) Metaplasia




Question 2: A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed. Which of the following mechanisms best explains the lysis of RBCs? (A) RBCs are biconcave in shape (B) RBCs are lysed because Osmo > Osmi (C) RBCs cannot pump Na+ out effectively (D) RBCs do not have a nucleus (E) RBCs take up the hypotonic solution too rapidly through pinocytosis




Question 3: A 55 year old hypertensive obese man presents with sudden weakness of the right side of his body. He is subsequently diagnosed to have had a stroke. Which is the correct order of the progression of events? (A) Embolism, thrombos

In [22]:
import docx
import msoffcrypto
import io


def docxReader(path, password=None):
    if password is None:
        try:
            doc = docx.Document(path)
            content = [para.text for para in doc.paragraphs]
            if not content:
                return "The document is empty or failed to read content."
            return "\n".join(content)
        except Exception as e:
            return f"Failed to read the document: {str(e)}"
    else:
        try:
            with open(path, "rb") as file:
                office_file = msoffcrypto.OfficeFile(file)
                office_file.load_key(password=password)
                decrypted = io.BytesIO()
                office_file.decrypt(decrypted)
                decrypted.seek(0)

                # Attempt to read the decrypted content with docx
                try:
                    # Use the 'docx.Document' method directly on the decrypted BytesIO stream
                    doc = docx.Document(decrypted)
                    content = [para.text for para in doc.paragraphs]
                    if not content:
                        return "The document is empty or failed to read content after decryption."
                    return "\n".join(content)
                except Exception as e:
                    return f"Failed to read the decrypted document: {str(e)}"

        except Exception as e:
            return f"Failed to read the document with the provided password: {str(e)}"


# Replace 'your_file.docx' with your actual file path
file_content = docxReader("./test_files/question.docx", "LKCexam")

print(file_content)  # Optional: Just to verify the content

In [6]:
import requests
import json
import re
# working formater
def modelcall(KEY, deploymentName, SYS_prompt, USER_prompt):
    ENDPOINT = f"https://intern2024openai.openai.azure.com/openai/deployments/{deploymentName}/chat/completions?api-version=2024-02-15-preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": KEY,
    }
    # Correct message structure
    message = [
        {"role": "system", "content": SYS_prompt},
        {"role": "user", "content": USER_prompt},
    ]
    payload = {
        "messages": message,
        "temperature": 0,
        "top_p": 0,
    }
    # Send request
    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status() 
        data = response.json()
    except requests.exceptions.RequestException as e:
        try:
            error_response = response.json()
            if "error" in error_response:
                content_filter_result = (
                    error_response["error"]
                    .get("innererror", {})
                    .get("content_filter_result", {})
                )
                filtered_items = {
                    k: v
                    for k, v in content_filter_result.items()
                    if v["severity"] != "safe"
                }

                if filtered_items:
                    error_message = (
                        "Filtered due to the following reasons:"
                    )
                    for item, details in filtered_items.items():
                        error_message += f" ╽{item.capitalize()}: severity = {details['severity']}, filtered = {details['filtered']}╽"
                    return ["error", "filtered", error_message]
                else:
                    return [
                        "error", 
                        "filtering",
                        "An error occurred, but no content filter issues were found.",
                    ]
            else:
                return [
                    "error",
                    "no_details",
                    "An error occurred, but no detailed error information is available. Goodluck",
                ]
        except json.JSONDecodeError:
            return ["error", "json_decode", f"Request error: {e}"]
        except Exception as e:
            return ["error", "unexpected", f"An unexpected error occurred: {e}"]

    content = data["choices"][0]["message"]["content"]
    pattern = r"```json(.*?)```"
    match = re.search(pattern, content, re.DOTALL)
    if match:
        try:
            return ["success", json.loads(match.group(1).strip())]
        except json.JSONDecodeError as e:
            return ["error", "json_parse", f"JSON parsing error: {e}\n\nContent: {content}"]
    else:
        try:
            return ["success", json.loads(content)]
        except json.JSONDecodeError as e:
            return [
                "error",
                "json_parse",
                f"assistant_response: {content}"
            ]
        except Exception as e:
            return ["error", "unexpected", f"An unexpected error occurred: {e}"]


USER_prompt = input("Hi, What powerpoint content would you like to generate:\n")
SYS_prompt = """Instruct:
You are to take the user question and seperate it into 3 sections labeled as "stem", "lead-in", "options" or 2 section labeled as "lead-in", "options" if "stem" is not present.

The "stem" is the main body of the question, providing the context or scenario. The "lead-in" is the specific question or directive that follows the "stem", indicating what the examinee needs to determine or answer. The "options" are the multiple-choice answers provided for the examinee to select from.

If unable to extract from a question leave "status" as "fail". "status" is only "success" if content of "lead-in" is not blank and "options" is not an empty.

There can be more then one question.

Guideline:
return in json format having the question id, status, data (stem, lead-in, options).

Example:
Question 1:	Which of the following adaptions to cell injury is described as an increased number of cells within an organ?	

	(A)	Atrophy
	(B)	Dysplasia
	(C)	Hyperplasia
	(D)	Hypertrophy
	(E)	Metaplasia
		
Question 2: DNA replication is known to be semi-conservative. Which of the following statements best describes this semi-conservative nature?

Question 3:	A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed. Which of the following mechanisms best explains the lysis of RBCs?

	(A)	RBCs are biconcave in shape
	(B)	RBCs are lysed because Osmo > Osmi
	(C)	RBCs cannot pump Na+ out effectively
	(D)	RBCs do not have a nucleus
	(E)	RBCs take up the hypotonic solution too rapidly through pinocytosis


is returned as

[
    {
        "id": 1,
        "status": "success",
        "data": [{
            "stem": "",
            "lead-in": "Which of the following adaptions to cell injury is described as an increased number of cells within an organ?",
            "options": {
                "A": "Atrophy",
                "B": "Dysplasia",
                "C": "Hyperplasia",
                "D": "Hypertrophy",
                "E": "Metaplasia"
            }
        }]
    },
    {
        "id": 2,
        "status": "fail",
        "data": [{
            "stem": "DNA replication is known to be semi-conservative.",
            "lead-in": "Which of the following statements best describes this semi-conservative nature?",
            "options": {}
            }]
    },
    {
        "id": 3,
        "status": "success",
        "data": [{
            "stem": "A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.",
            "lead-in": "Which of the following mechanisms best explains the lysis of RBCs?",
            "options": {
                "A": "RBCs are biconcave in shape",
                "B": "RBCs are lysed because Osmo > Osmi",
                "C": "RBCs cannot pump Na+ out effectively",
                "D": "RBCs do not have a nucleus",
                "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
            }
        }]
    }
]
"""

['success',
 [{'id': 1,
   'status': 'success',
   'data': [{'stem': 'Which of the following adaptions to cell injury is described as an increased number of cells within an organ?',
     'lead-in': '',
     'options': {'A': 'Atrophy',
      'B': 'Dysplasia',
      'C': 'Hyperplasia',
      'D': 'Hypertrophy',
      'E': 'Metaplasia'}}]},
  {'id': 2,
   'status': 'success',
   'data': [{'stem': 'A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.',
     'lead-in': 'Which of the following mechanisms best explains the lysis of RBCs?',
     'options': {'A': 'RBCs are biconcave in shape',
      'B': 'RBCs are lysed because Osmo > Osmi',
      'C': 'RBCs cannot pump Na+ out effectively',
      'D': 'RBCs do not have a nucleus',
      'E': 'RBCs take up the hypotonic solution too rapid

```json
['success',
 [{'id': 1,
   'status': 'success',
   'data': [{'stem': 'Which of the following adaptions to cell injury is described as an increased number of cells within an organ?',
     'lead-in': '',
     'options': {'A': 'Atrophy',
      'B': 'Dysplasia',
      'C': 'Hyperplasia',
      'D': 'Hypertrophy',
      'E': 'Metaplasia'}}]},
  {'id': 2,
   'status': 'success',
   'data': [{'stem': 'A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.',
     'lead-in': 'Which of the following mechanisms best explains the lysis of RBCs?',
     'options': {'A': 'RBCs are biconcave in shape',
      'B': 'RBCs are lysed because Osmo > Osmi',
      'C': 'RBCs cannot pump Na+ out effectively',
      'D': 'RBCs do not have a nucleus',
      'E': 'RBCs take up the hypotonic solution too rapidly through pinocytosis'}}]},
  {'id': 3,
   'status': 'success',
   'data': [{'stem': 'A 55 year old hypertensive obese man presents with sudden weakness of the right side of his body. He is subsequently diagnosed to have had a stroke.',
     'lead-in': 'Which is the correct order of the progression of events?',
     'options': {'A': 'Embolism, thrombosis, ischaemia, endothelial injury',
      'B': 'Endothelial injury, embolism, thrombosis, ischaemia',
      'C': 'Endothelial injury, thrombosis, embolism, ischaemia',
      'D': 'Ischaemia, thrombosis, endothelial injury, embolism',
      'E': 'Thrombosis, embolism, endothelial injury, ischaemia'}}]}]]
```

In [ ]:
modelcall(
    KEY=get_key(),
    deploymentName="gpt-35-thread1",
    SYS_prompt=SYS_prompt,
    USER_prompt=USER_prompt,
)

In [ ]:
import os
import requests
import base64
import re

# Configuration
KEY = get_key()

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("ascii")
user_input = input("Hi, What powerpoint content would you like to generate:\n")
SYS_prompt = """Instruct:
You are to take the user question and seperate it into 3 sections labeled as stem, lead-in, options or 2 section labeled as lead-in, options if stem is not present.

The stem is the main body of the question, providing the context or scenario. The lead-in is the specific question or directive that follows the stem, indicating what the examinee needs to determine or answer. The options are the multiple-choice answers provided for the examinee to select from.

Guideline:
return in a json format

Example:
Question 2:	A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.

Which of the following mechanisms best explains the lysis of RBCs?

	(A)	RBCs are biconcave in shape
	(B)	RBCs are lysed because Osmo > Osmi
	(C)	RBCs cannot pump Na+ out effectively
	(D)	RBCs do not have a nucleus
	(E)	RBCs take up the hypotonic solution too rapidly through pinocytosis

is returned as

{
  "stem": "A clinician-scientist was isolating leukocytes for a biomedical assay. He successfully removed red blood cells (RBCs) from other blood cells by briefly exposing the cell suspension to a hypotonic solution. Other blood cells remained intact while RBCs were lysed.",
  "lead_in": "Which of the following mechanisms best explains the lysis of RBCs?",
  "options": {
    "A": "RBCs are biconcave in shape",
    "B": "RBCs are lysed because Osmo > Osmi",
    "C": "RBCs cannot pump Na+ out effectively",
    "D": "RBCs do not have a nucleus",
    "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
  }
}

"""

headers = {
    "Content-Type": "application/json",
    "api-key": KEY,
}

# Payload for the request
payload = {
    "messages": [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": SYS_prompt,
                }
            ],
        },
        {"role": "user", 
         "content": [{"type": "text", "text": user_input}]},
    ],
    "temperature": 0.7,
    "top_p": 0.95,
}

ENDPOINT = "https://intern2024openai.openai.azure.com/openai/deployments/gpt4-Omni/chat/completions?api-version=2024-02-15-preview"


# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")
data = response.json()

def extract_json_text(text):
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None


print(data["choices"][0]["message"]["content"])

{
  "lead_in": "Which of the following mechanisms best explains the lysis of RBCs?",
  "options": {
    "A": "RBCs are biconcave in shape",
    "B": "RBCs are lysed because Osmo > Osmi",
    "C": "RBCs cannot pump Na+ out effectively",
    "D": "RBCs do not have a nucleus",
    "E": "RBCs take up the hypotonic solution too rapidly through pinocytosis"
  }
}


In [ ]:
import openai


def chat_with_gpt(
    api_key, system_prompt, user_prompt, history=None, use_system_prompt=True
):
    openai.api_key = api_key

    if history is None:
        history = []

    # Prepare the message history
    messages = []
    if use_system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages += history
    messages.append({"role": "user", "content": user_prompt})

    # Call OpenAI API
    response = openai.ChatCompletion.create(model="gpt-4", messages=messages)

    # Get the assistant's reply
    assistant_reply = response["choices"][0]["message"]["content"]

    # Update the history
    new_history = history + [
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": assistant_reply},
    ]

    return [assistant_reply, new_history]


# Example usage:
api_key = "your_openai_api_key"
system_prompt = "You are a helpful assistant."
user_prompt = "Hello, how are you?"
history = None

# Initial call with system prompt
response = chat_with_gpt(api_key, system_prompt, user_prompt, history)
print("Assistant Reply:", response[0])
print("Full History:", response[1])

# Subsequent call without system prompt
new_user_prompt = "What can you do?"
response = chat_with_gpt(
    api_key, system_prompt, new_user_prompt, response[1], use_system_prompt=False
)
print("Assistant Reply:", response[0])
print("Full History:", response[1])